In [1]:
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
from keras.models import load_model
# Önceki eğittiğiniz modeli yükleyin
eski_model_path = "cnn_epoch_100.h5"
eski_model = load_model(eski_model_path)

# CNN modelini oluşturun
cnn_model = Sequential()
cnn_model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
cnn_model.add(Dense(4, activation='softmax'))  # Sınıflandırma tipinize göre ayarlayın


In [ ]:
# Önceki modeli ve CNN modelini birleştirin
birlesik_model = Sequential()
birlesik_model.add(eski_model)
birlesik_model.add(cnn_model)

In [ ]:
# Yeni veri setini yükleyin ve ön işleme yapın
yeni_veri_path = "Split Data/1-20/1-20-Train/"
image_size = (224, 224)
batch_size = 4

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
        rescale=1./255,
        horizontal_flip=True,
        validation_split=0.2,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
) # %70 eğitim datası içinden ön eğitim kontrölü içinde %20 yani 140 adet ön test (daha önceden gördüğü veri üzerinden test yapmak için) veri ayırıyoruz

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=yeni_veri_path,
    x_col='Filepath',
    labels='inferred',
    label_mode='int',
    y_col='Label',
    target_size=image_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=123,
    cros_to_aspect_ratio =False,
    follow_links=False,
    interpolation='bilinear',
    subset='training'
)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=yeni_veri_path,
    x_col='Filepath',
    label_mode='int',
    labels='inferred',
    y_col='Label',
    target_size=image_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    interpolation='bilinear',
    shuffle=True,
    seed=123,
    follow_links=False,
    cros_to_aspect_ratio =False,
    subset='validation'
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=yeni_veri_path,
    x_col='Filepath',
    label_mode='int',
    labels='inferred',
    y_col='Label',
    target_size=image_size,
    color_mode='rgb',
    class_mode='categorical',
    interpolation='bilinear',
    batch_size=batch_size,
    cros_to_aspect_ratio =False,
    follow_links=False,
    shuffle=False
)

In [ ]:
# Modeli yeni veri setiyle eğitin
epochs = 100
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
birlesik_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

In [ ]:
birlesik_model.fit(yeni_veri, epochs=epochs,callbacks=[callbacks])

In [ ]:
# Güncellenmiş modeli kaydedin
guncellenmis_model_path = "guncellenmis_cnn_epoch_100.h5"
birlesik_model.save(guncellenmis_model_path)